In [29]:
import psycopg2
import pandas as pd

# Prepare the Database

Run `docker-compose up` to get Postgres up and running.

In [30]:
credentials = "dbname='astronomy' user='user' host='localhost' password='pass'"
conn = psycopg2.connect(credentials)
cur = conn.cursor()

cur.execute("""
DROP TABLE IF EXISTS Star;

CREATE TABLE Star (
  kepler_id INTEGER PRIMARY KEY,
  t_eff INTEGER,
  radius FLOAT
);


COPY Star (kepler_id, t_eff, radius) FROM '/data/data1/stars.csv' CSV;
""")
conn.commit()

# Task 1
As a warm-up, add the following stars to the existing Star table:

| kepler_id | t_eff | radius |
|-----------|-------|--------|
| 7115384   | 3789  | 27.384 |
| 8106973   | 5810  | 0.811  |
| 9391817   | 6200  | 0.958  |

In [31]:
cur.execute("""
INSERT INTO Star (kepler_id, t_eff, radius) VALUES
  (7115384, 3789, 27.384),
  (8106973, 5810, 0.811),
  (9391817, 6200, 0.958);
""")
conn.commit()

# Add Planet Table

In [32]:
cur.execute("""
DROP TABLE IF EXISTS Planet;

CREATE TABLE Planet (
  kepler_id INTEGER NOT NULL,
  koi_name VARCHAR(20) NOT NULL,
  kepler_name VARCHAR(20),
  status VARCHAR(20) NOT NULL,
  period FLOAT,
  radius FLOAT,
  t_eq INTEGER
);


COPY Planet (kepler_id, koi_name, kepler_name, status, period, radius, t_eq)
  FROM '/data/data1/planets.csv' CSV;
""")
conn.commit()

# Task 2

Your task is to fix a `Planet` table has been corrupted.

There are two problems: some of the rows were copied with a negative radius
and the unconfirmed planets were given fake Kepler names. To fix this, you'll
have to:

- Update the Kepler names of planets which don't have a confirmed status
  (replace the `kepler_name` with `NULL`);
- Delete rows where the `radius` is negative.
- Have a look at the table to find out what needs to be changed.

In [33]:
cur.execute("""
UPDATE Planet SET kepler_name = NULL WHERE status != 'CONFIRMED';
DELETE FROM Planet WHERE radius < 0;
""")
conn.commit()

# Task 2

Now you've ready to create your first table. Your task is to set up a new
`Planet` table and fill it with the planets listed below.

Your table should consist of the following attributes in this order:

- `kepler_id`, as type `INTEGER`
- `koi_name`, as type `VARCHAR(15)`
- `kepler_name`, as type `VARCHAR(15)`
- `status`, as type `VARCHAR(20)`
- `radius`, as type `FLOAT`

Add the following constraints:

- make all attributes `NOT NULL` except `kepler_name`;
- make the `koi_name` a unique attribute.

Finally, insert the following planets into your new table:

| kepler_id | koi_name  | kepler_name  | status    | radius  |
|-----------|-----------|--------------|-----------|---------|
| 6862328   | K00865.01 | NULL         | CANDIDATE | 119.021 |
| 10187017  | K00082.05 | Kepler-102 b | CONFIRMED | 5.286   |
| 10187017  | K00082.04 | Kepler-102 c | CONFIRMED | 7.071   |

Don't forget to use `NULL` for the missing planet name!

In [34]:
cur.execute("""
DROP TABLE IF EXISTS Planet;

CREATE TABLE Planet (
    kepler_id INTEGER NOT NULL,
    koi_name VARCHAR(15) NOT NULL UNIQUE,
    kepler_name VARCHAR(15),
    status VARCHAR(20) NOT NULL,
    radius FLOAT NOT NULL
);
INSERT INTO Planet VALUES
  (6862328, 'K00865.01', NULL, 'CANDIDATE', 119.021),
  (10187017, 'K00082.05', 'Kepler-102 b', 'CONFIRMED', 5.286),
  (10187017, 'K00082.04', 'Kepler-102 c', 'CONFIRMED', 7.071);
""")
conn.commit()